In [1]:
import numpy as np
import pandas as pd
import torch
import os

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from memotion_utility import load_data
from tqdm import tqdm

2024-07-04 21:56:14.161971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 21:56:14.162091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 21:56:14.286695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
CSV_FILE = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
LR = 2e-5
batch_size = 32
epochs = 20
seed = 123
downsample = True # whether we downsample training data to reduce bias

In [3]:
# path = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
# df = pd.read_csv(path)
# df.head()

In [4]:
# df.loc[df['text_corrected'].isna()] = 'nan'

In [5]:
# # Define the columns of interest
# columns_of_interest = ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']

# # Count occurrences of each value in specified columns
# value_counts = {}
# for column in columns_of_interest:
#     value_counts[column] = df[column].value_counts()

# # Display value counts
# for column, counts in value_counts.items():
#     print(f"Value counts for column '{column}':")
#     print(counts)
#     print()

In [6]:
# Path to the image folder
# image_folder_path = "/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/images"
# image_path = []
# for filename in os.listdir(image_folder_path):
#     image_path.append(f"{image_folder_path}/{filename}")
# print(len(image_path))

In [7]:
# n = 99
# image = Image.open(image_path[n]).convert('RGB')
# image

In [8]:
# image_path[n]

In [9]:
# df[df["image_name"]=="image_6297.jpg"]

In [10]:
# # Define the function to map value
# def map_values(value):
#     if value.lower() in ['funny', 'very_funny', 'hilarious']:
#         return 1
#     elif value.lower() in ['general', 'twisted_meaning', 'very_twisted']:
#         return 1
#     elif value.lower() in ['slight', 'very_offensive', 'hateful_offensive']:
#         return 1
#     elif value.lower() == 'motivational':
#         return 1
#     else:
#         return 0

# columns_to_map = ['humour', 'sarcasm', 'offensive', 'motivational']
# df[columns_to_map] = df[columns_to_map].applymap(map_values)

# df


In [11]:
# for column in ['humour', 'sarcasm', 'offensive', 'motivational']:
#     if df[column].dtype == 'object':
#         print(f"Value Counts for {column}:")
#         print(df[column].value_counts())
#         print("\n")

# # Visualize the distribution of each mapped column
# plt.figure(figsize=(12, 8))
# for i, column in enumerate(['humour', 'sarcasm', 'offensive', 'motivational']):
#     plt.subplot(2, 2, i+1)
#     sns.countplot(data=df, x=column)
#     plt.title(f"{column.capitalize()} Distribution")
#     plt.xlabel("Value")
#     plt.ylabel("Count")
# plt.tight_layout()
# plt.show()

In [12]:
# value_counts = {}
# for column in columns_of_interest:
#     value_counts[column] = df[column].value_counts()
    
# # Display value counts
# for column, counts in value_counts.items():
#     print(f"Value counts for column '{column}':")
#     print(counts)
#     print()

In [13]:
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [14]:
train_df,val_df,test_df = load_data(CSV_FILE,downsample = downsample)

train : 
 label
1    1953
0    1953
Name: count, dtype: int64
val : 
 label
1    343
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    543
Name: count, dtype: int64


In [15]:
class Memotion_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        label = self.data.iloc[idx]['label']
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
train_dataset = Memotion_Dataset(train_df, tokenizer)
val_dataset = Memotion_Dataset(val_df, tokenizer)
test_dataset = Memotion_Dataset(test_df, tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# optimizer = AdamW(model.parameters(), lr=2e-5)
# epochs = 5

# for epoch in range(epochs):
#     model.train()
#     for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}'):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#     # Evaluation
#     model.eval()
#     correct = 0
#     total = 0
#     all_labels = []
#     all_predictions = []
#     for batch in tqdm(val_loader, desc=f'Epoch {epoch + 1} - Validation'):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)
#         with torch.no_grad():
#             outputs = model(input_ids, attention_mask=attention_mask)
#             logits = outputs.logits
#             predictions = torch.argmax(logits, dim=1)
#             correct += (predictions == labels).sum().item()
#             total += labels.size(0)
#             all_labels.extend(labels.cpu().numpy())
#             all_predictions.extend(predictions.cpu().numpy())

#     accuracy = accuracy_score(all_labels, all_predictions)
#     precision = precision_score(all_labels, all_predictions, average='macro')
#     recall = recall_score(all_labels, all_predictions, average='macro')
#     f1 = f1_score(all_labels, all_predictions, average='macro')
#     conf_matrix = confusion_matrix(all_labels, all_predictions)
#     class_report = classification_report(all_labels, all_predictions, target_names=['Not Offensive', 'Offensive'])

#     # Print metrics
#     print(f"Accuracy: {accuracy * 100:.2f}%")
#     print(f"Precision: {precision:.2f}")
#     print(f"Recall: {recall:.2f}")
#     print(f"F1 Score: {f1:.2f}")
#     print("Confusion Matrix:")
#     print(conf_matrix)
#     print("Classification Report:")
#     print(class_report)

In [19]:
# # Test evaluation
# model.eval()
# correct = 0
# total = 0
# all_labels = []
# all_predictions = []
# for batch in tqdm(test_loader, desc='Test Evaluation'):
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     labels = batch['label'].to(device)
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=1)
#         correct += (predictions == labels).sum().item()
#         total += labels.size(0)
#         all_labels.extend(labels.cpu().numpy())
#         all_predictions.extend(predictions.cpu().numpy())

# accuracy = accuracy_score(all_labels, all_predictions)
# precision = precision_score(all_labels, all_predictions, average='macro')
# recall = recall_score(all_labels, all_predictions, average='macro')
# f1 = f1_score(all_labels, all_predictions, average='macro')
# conf_matrix = confusion_matrix(all_labels, all_predictions)
# class_report = classification_report(all_labels, all_predictions, target_names=['Not Offensive', 'Offensive'])

# # Print metrics
# print("Test Set Metrics")
# print(f"Accuracy: {accuracy * 100:.2f}%")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1 Score: {f1:.2f}")
# print("Confusion Matrix:")
# print(conf_matrix)
# print("Classification Report:")
# print(class_report)

In [20]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = np.argmax(preds, axis=-1)

    accuracy = accuracy_score(labels,preds) 
    # Compute precision, recall, and F1 score for both classes and macro average
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro')
    
    result = {
        "accuracy": accuracy,
        "precision_not_offensive": round(precision[0] * 100, 4),
        "recall_not_offensive": round(recall[0] * 100, 4),
        "f1_not_offensive": round(f1[0] * 100, 4),
        "precision_offensive": round(precision[1] * 100, 4),
        "recall_offensive": round(recall[1] * 100, 4),
        "f1_offensive": round(f1[1] * 100, 4),
        "precision_macro": round(precision_macro * 100, 4),
        "recall_macro": round(recall_macro * 100, 4),
        "f1_macro": round(f1_macro * 100, 4),
    }
    
    return result


In [21]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate = LR,
    seed = seed,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate(eval_dataset = test_dataset)

# Print the results
print(evaluation_results)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Not Offensive,Recall Not Offensive,F1 Not Offensive,Precision Offensive,Recall Offensive,F1 Offensive,Precision Macro,Recall Macro,F1 Macro
1,No log,0.724536,0.389286,38.819300,100.000000,55.927800,100.000000,0.291500,0.581400,69.409700,50.145800,28.254600
2,No log,0.706524,0.451786,36.526900,56.221200,44.283100,57.964600,38.192400,46.045700,47.245800,47.206800,45.164400
3,No log,0.746624,0.457143,35.254200,47.926300,40.625000,57.358500,44.314900,50.000000,46.306400,46.120600,45.312500
4,No log,0.888215,0.433929,33.221500,45.622100,38.446600,54.961800,41.982500,47.603300,44.091700,43.802300,43.025000
5,0.633000,1.176225,0.467857,35.789500,47.004600,40.637500,58.181800,46.647200,51.779900,46.985600,46.825900,46.208700
6,0.633000,1.688159,0.458929,35.761600,49.769600,41.618500,57.751900,43.440200,49.584000,46.756800,46.604900,45.601300
7,0.633000,1.995271,0.496429,36.051500,38.709700,37.333300,59.327200,56.559800,57.910400,47.689400,47.634700,47.621900
8,0.633000,2.876354,0.425000,34.870300,55.760400,42.907800,54.929600,34.110800,42.086300,44.899900,44.935600,42.497100
9,0.163900,2.998171,0.476786,38.050300,55.760400,45.233600,60.330600,42.565600,49.914500,49.190400,49.163000,47.574100
10,0.163900,3.369160,0.442857,36.389700,58.525300,44.876300,57.346000,35.277000,43.682300,46.867800,46.901200,44.279300


{'eval_loss': 3.475162982940674, 'eval_accuracy': 0.5175125089349535, 'eval_precision_not_offensive': 40.8333, 'eval_recall_not_offensive': 54.1436, 'eval_f1_not_offensive': 46.5558, 'eval_precision_offensive': 63.3284, 'eval_recall_offensive': 50.2336, 'eval_f1_offensive': 56.0261, 'eval_precision_macro': 52.0809, 'eval_recall_macro': 52.1886, 'eval_f1_macro': 51.2909, 'eval_runtime': 6.6675, 'eval_samples_per_second': 209.824, 'eval_steps_per_second': 6.599, 'epoch': 20.0}
